<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Deep Learning
Your first model took random shots and your second model tried matching the current game against past games; however, if no past games matched then it just randomly guessed. This model will do better by __learning__ the pattern of ship layouts, not just memorizing them. 

## Data
We will be using past game date to train out model. First we need to download the data from the master game archive by running: (note the data is constantly being updated as games are played)
```bash
./bin/download_data.sha # Download, parse data, create sample data
```
next copy the sample data file into the mock directory of the code for this model
```bash
cp ./data/data.min.json ./containers/Shoot/cnn/mock/data/data.json
```
You will now be able to test your model training localy 

Next we need to upload the data to our s3 bucket so we can train the production model on it. run:
```bash
./bin/upload_data.sh
```

## Code
The code is in /containers/shoot/cnn. See [Endpoint](../Endpoint_Reference.ipynb) for details on each file.  In addition to running our model it will also put a tag in the session attribute that we will use in athena to compare different models. You can change the tag value (which you should do for each model) by editing the host.py file. Next, run the test script localy to confirm the scripts works.


## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in /shoot-config.js. you need to edit the following field to the following values

|parameter field|Value|description|
|---|---|---|
|trainsourcefile|`shoot-CNN.tar.gz`| code for our model|
|hostsourcefile|`shoot-CNN.tar.gz`|a code for our model|
|hyperparameters.epochs|10|Max epochs to train for|
|hyperparameters.warm_up|5|How many epochs to wait to start early stoppping|
|hyperparameters.patience|5|How many epochs with no improvement before early stopping|
|hyperparameters.learning_rate|0.005||
|hyperparameters.width|32|How many feature filters in each layer|
|hyperparameters.depth|8|how many layers|
|hyperparameters.batch_size|1024|batch size for training|
|metrics|[{Name:"Validation",Regex:"Testing loss: (.*?);"},{Name:"Training",Regex:"Training loss: (.*?);"},{Name:"Throughput",Regex:"Throughput=(.*?);"}]|Metrics to send to cloudwatch logs|
|traininstancetype|"ml.p3.2xlarge"|instance type to train with|
|channels|{"train":{"path":"all/data.json"}}|path to data in data bucket|

## Deploy
1. follow instructions from endpoint reference to commit your code and then push your changes to the remote branch.
2. tell opperations to deploy your changes
3. Monitor you models performance. see [Athena](../Athena.ipynb)

## Next Steps
try editing hyperparamters or the code to get better results